In [5]:
import pandas as pd    
import geopandas as gpd
from geopandas.tools import geocode
import numpy as np
import scipy.stats as stats
import scipy
import seaborn as sns
import matplotlib 
from matplotlib import pyplot as plt
matplotlib.rcParams.update({'font.size': 20})

In [6]:
import urllib.request as request

**Imports all traffic data from 2007 - 2018**

Link: https://data.seattle.gov/browse?q=traffic%20flow%20count&sortBy=relevance to download yearly data

In [7]:
url_list = ['https://opendata.arcgis.com/datasets/7015d5d46a284f94ac05c2ea4358bcd7_0.geojson',
            'https://opendata.arcgis.com/datasets/5fc63b2a48474100b560a7d98b5097d7_1.geojson',
            'https://opendata.arcgis.com/datasets/27af9a2485c5442bb061fa7e881d7022_2.geojson',
            'https://opendata.arcgis.com/datasets/4f62515558174f53979b3be0335004d3_3.geojson',
            'https://opendata.arcgis.com/datasets/29f801d03c9b4b608bca6a8e497278c3_4.geojson',
            'https://opendata.arcgis.com/datasets/a0019dd0d6464747a88921f5e103d509_5.geojson',
            'https://opendata.arcgis.com/datasets/40bcfbc4054549ebba8b5777bbdd40ff_6.geojson',
            'https://opendata.arcgis.com/datasets/16cedd233d914118a275c6510115d466_7.geojson',
            'https://opendata.arcgis.com/datasets/902fd604ecf54adf8579894508cacc68_8.geojson',
            'https://opendata.arcgis.com/datasets/170b764c52f34c9497720c0463f3b58b_9.geojson',
            'https://opendata.arcgis.com/datasets/2c37babc94d64bbb938a9b520bc5538c_10.geojson',
            'https://opendata.arcgis.com/datasets/a35aa9249110472ba2c69cc574eff984_11.geojson']
    

In [28]:
def get_gdf(year):
    '''Enter the desired year to download the traffic flow count
    data for that year. Example: enter '7' for the year 2007.
    '''
    num = year-7
    gdf_year = gpd.read_file(url_list[num])
    if year == 11:
        gdf_year = gdf_year.rename(columns={"YEAR_" : 'YEAR'})
        gdf_year = gdf_year[gdf_year.STNAME != '16TH AVE S']
    if year == 12:
        gdf_year = gdf_year.rename(columns={'STDY_YEAR' : 'YEAR'})
        gdf_year = gdf_year[gdf_year.YEAR.notnull()]
    if year == 15 or year == 16:
        gdf_year = gdf_year.rename(columns={"COUNTAAWDT" : 'AAWDT', "FLOWSEGID" : "GEOBASID", 'FIRST_STNAME_ORD' : 'STNAME'})
        gdf_year = gdf_year[['AAWDT', 'GEOBASID', 'STNAME', 'SHAPE_Length', 'geometry']]
        if year == 15:
            year_list = [2015]*len(gdf_year)
            gdf_year['YEAR'] = year_list
        elif year == 16:
            year_list = [2016]*len(gdf_year)
            gdf_year['YEAR'] = year_list
    elif year == 17 or year == 18:
        gdf_year = gdf_year.rename(columns={"AWDT" : 'AAWDT', "FLOWSEGID" : "GEOBASID", 'STNAME_ORD' : 'STNAME'})
        gdf_year = gdf_year[['AAWDT', 'GEOBASID', 'STNAME', 'SHAPE_Length', 'geometry']]
        if year == 17:
            year_list = [2017]*len(gdf_year)
            gdf_year['YEAR'] = year_list
        elif year == 18:
            year_list = [2018]*len(gdf_year)
            gdf_year['YEAR'] = year_list
    #df_year_AAWDT = df_year['AAWDT'].values
    #df_year_geobase = df_year['GEOBASID'].values
    #df_year_dist = df_year['SHAPE_Length'].values
    gdf_year = gdf_year[[ 'YEAR', 'AAWDT', 'GEOBASID', 'STNAME', 'SHAPE_Length', 'geometry']]
    gdf_year = gdf_year[gdf_year.YEAR != 0]
    gdf_year = gdf_year[gdf_year.YEAR.notnull()]
    return gdf_year #, df_year_AAWDT, df_year_geobase, df_year_dist

In [24]:
gdf_12 = get_gdf(12)

In [25]:
gdf_12['YEAR'].unique()

array([2012.])

In [29]:
year_list = [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
agg_year_data = gpd.GeoDataFrame()
for year in year_list:
    agg_year_data = agg_year_data.append(get_gdf(year))
agg_year_data.head()

,YEAR,AAWDT,GEOBASID,STNAME,SHAPE_Length,geometry
0,2007.0,7200.0,94250370.0,RENTON AVE S,4615.134504,"LINESTRING (-122.28064 47.53541, -122.28056 47..."
1,2007.0,13100.0,50000650.0,AIRPORT WAY S,18205.478931,"MULTILINESTRING ((-122.29348 47.52408, -122.29..."
2,2007.0,28300.0,127150040.0,N NORTHGATE WAY,4267.857028,"LINESTRING (-122.34469 47.70505, -122.34334 47..."
3,2007.0,8200.0,123450090.0,N 65TH ST,12341.641545,"LINESTRING (-122.34831 47.67606, -122.34990 47..."
4,2007.0,12400.0,9550200.0,2ND AVE,4161.475805,"LINESTRING (-122.34045 47.61096, -122.34165 47..."


In [30]:
agg_year_data['YEAR'].unique()

array([2007., 2008., 2009., 2010., 2011., 2012., 2013., 2014., 2015.,
       2016., 2017., 2018.])

In [8]:
import altair as alt

In [9]:
alt.Chart(agg_year_data).mark_bar().encode(
    alt.X('Year'),
    y='AAWdT'
    # could also use alt.Y(aggregate='count', type='quantitative')
)

MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000). For information on how to plot larger datasets in Altair, see the documentation

alt.Chart(...)

In [17]:
montlake_br_traffic_07 = df_07.loc[df_07['STNAME']=='MONTLAKE BR']

In [19]:
montlake_br_traffic_07

,OBJECTID,GEOBASID,DOWNTOWN,COMPKEY,STNAME,OID_,NAME,YEAR,SEGKEY,AAWDT,INPUT_STUD,HPMS,FLOW_COUNT,SHAPE_Length
197,198,86850010,N,11902,MONTLAKE BR,83,"MONTLAKE BR, S/O POINT A",2007,11902,57203,0,,Y,1362.081215


In [49]:
df_048 = get_df(8)

In [10]:
def get_st_data(year_list, stname_list):
    '''Enter a list of the desired years and street name (as a list of strings) to download the traffic flow count
    data for that year. Example: enter '7' for the year 2007. Make sure to enter a list even if only requesting a single 
    year or street name.'''
    all_data = gpd.GeoDataFrame()
    for i in range(len(year_list)):
        year = year_list[i]
        gdf_year = get_gdf(year)
        all_data = all_data.append(gdf_year)
    st_data_year = gpd.GeoDataFrame()
    for j in range(len(stname_list)):
        st_data_year = st_data_year.append(all_data.loc[all_data['STNAME'] == stname_list[j]])
    return st_data_year

In [129]:
montlake_br_data_08 = get_st_data([8, 12], ['MONTLAKE BLVD NE', 'MONTLAKE BR'])

In [130]:
montlake_br_data_08

,YEAR,AAWDT,GEOBASID,STNAME,SHAPE_Length,geometry
149,2008.0,46100.0,86950030.0,MONTLAKE BLVD NE,3751.859475,"LINESTRING (-122.30459 47.64900, -122.30458 47..."
108,2012.0,42312.0,86950030.0,MONTLAKE BLVD NE,2966.829559,"LINESTRING (-122.30391 47.65108, -122.30345 47..."
198,2008.0,57100.0,86850010.0,MONTLAKE BR,1362.081215,"LINESTRING (-122.30462 47.64526, -122.30461 47..."
140,2012.0,55764.0,86850010.0,MONTLAKE BR,2147.111130,"LINESTRING (-122.30462 47.64526, -122.30461 47..."


In [11]:
montlake_blvd_allyear = get_st_data(year_list, ['MONTLAKE BLVD NE'])

In [14]:
montlake_blvd_allyear.describe()

,AAWDT,GEOBASID,SHAPE_Length
count,24.000000,2.400000e+01,24.000000
mean,40428.816917,2.898427e+07,1989.032606
std,5569.503125,4.186954e+07,1430.923947
min,31500.000000,1.312000e+03,495.856467
25%,36000.000000,1.412000e+03,575.275702
50%,40906.000000,1.425500e+03,1875.929737
75%,44925.000000,8.695003e+07,3163.087038
max,48400.000000,8.695003e+07,3751.859475


In [21]:
alt.Chart(montlake_blvd_allyear).mark_bar().encode(
    alt.X('YEAR:O'),
    y ='AAWDT:O'
    # could also use alt.Y(aggregate='count', type='quantitative')
)

alt.Chart(...)